In [1]:
import pandas as pd
import numpy as np 
# import plotly
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords 
from nltk.stem import PorterStemmer 
from nltk.probability import FreqDist
from os import path 
from PIL import Image 
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator 
import matplotlib.pyplot as plt 
import plotly.express as px
%matplotlib inline


# Data Cleaning

In [2]:
house_permit_from_file = pd.read_csv('Housing_Permits.csv',dtype = 'unicode')

In [3]:
house_permit_from_file = house_permit_from_file.filter(items = ['PropertyAddress','PermitDescription','Neighborhood'])

In [4]:
# Question 1 What types of construction permits are people pulling?
house_permit_from_file['PermitDescription'] = house_permit_from_file['PermitDescription'].astype(str)
house_permit_from_file = house_permit_from_file[house_permit_from_file['PermitDescription'].notnull()]

In [5]:
house_permit_from_file['lowercase'] = house_permit_from_file['PermitDescription'].str.lower()

In [6]:
house_permit_from_file['tokenized_text'] = house_permit_from_file['lowercase'].apply(word_tokenize)

In [7]:
house_permit_from_file_filter = house_permit_from_file.filter(items = ['PropertyAddress','lowercase','tokenized_text','Neighborhood']).reset_index()

In [8]:
stop_words=set(stopwords.words("english"))
house_permit_from_file_filter['no_stopwords'] = house_permit_from_file_filter['tokenized_text'].apply(lambda tokenized_list:[word for word in tokenized_list if word not in stop_words])

In [9]:
symbol_list = ["!", ")","(", ":", ",",".", "...","-", "|", "'",'"','’','…',"“","”","&","^",";","@","#","$","%","_","+","="]
stop_words.update(symbol_list)
house_permit_from_file_filter['no_symbols'] = house_permit_from_file_filter['no_stopwords'].apply(lambda x: [word for word in x if word not in stop_words])
house_permit_from_file_filter['no_symbols'] = house_permit_from_file_filter['no_symbols'].apply(lambda x: [word for word in x if not word.isnumeric()])

In [10]:
common_word_list = ['per','code','install','new','permit','work']
common_word_set = set(common_word_list)
house_permit_from_file_filter['remove_common_word'] = house_permit_from_file_filter['no_symbols'].apply(lambda x: [word for word in x if word not in common_word_set])

In [11]:
house_permit_from_file_filter['tokenized_final'] = house_permit_from_file_filter['no_stopwords'].apply(lambda x: [word for word in x if word not in stop_words])
house_permit_from_file_filter['tokenized_final'] = house_permit_from_file_filter['tokenized_final'].apply(lambda x: [word for word in x if not word.isnumeric()])
house_permit_from_file_filter['tokenized_final'] = house_permit_from_file_filter['tokenized_final'].apply(lambda x: [word for word in x if word not in common_word_set])

In [12]:
house_permit_analysis = house_permit_from_file_filter.filter(items = ['PropertyAddress','tokenized_final','Neighborhood'])

In [13]:
house_permit_analysis = house_permit_analysis.rename(columns = {'PropertyAddress':'address'})

In [14]:
house_permit_analysis.to_csv('house_permit_analysis.csv')

In [15]:
house_permit_analysis.head()

,address,tokenized_final,Neighborhood
0,1752 N GAY ST,"[interior, exterior, alteration, existing, bui...",BROADWAY EAST
1,3401 LABYRINTH RD,"[demolition, building/structure, wood, length:...",CROSS COUNTRY
2,815 BROOKS LANE,"[windows, 2-new, exterior, doors, ceramic, flo...",RESERVOIR HILL
3,1743 DRUID HILL AVE,"[*authorization, proceed*, sanitary, line, rea...",UPTON
4,6026 PINEHURST RD,"[plumbing, fixtures, sanitary, connection, rec...",LAKE EVESHAM


# Neighborhood Keyword Freq Analysis

In [16]:
neighborhoods = house_permit_analysis.groupby('Neighborhood').sum()['tokenized_final'].reset_index()

In [17]:
neighborhoods['list'] = neighborhoods['tokenized_final'].tolist()

In [18]:
description_list_final_n = house_permit_analysis['tokenized_final'].tolist()
word_list_final_n = []
for sublist in description_list_final_n:
    for word in sublist:
        word_list_final_n.append(word)
word_frequency_dict_final_n = FreqDist(word_list_final_n)

In [19]:
# Add frequency to the result but it doesn't mean anything
df_word_frequency_n = pd.DataFrame(word_frequency_dict_final_n.items(),columns=['word','frequency'])
df_word_frequency_n = df_word_frequency_n.sort_values(by = 'frequency', ascending = False).reset_index()
del df_word_frequency_n['index']
frequency_sum_n = df_word_frequency_n['frequency'].sum()
# print(frequency_sum)
df_word_frequency_n.head(20)

,word,frequency
0,interior,147207
1,replace,144060
2,existing,128984
3,btu,119145
4,original,114871
5,floor,107525
6,gas,106455
7,water,105438
8,inches,101077
9,plans,96810


In [20]:
neighborhoods.to_csv('neighborhoods.csv')

In [21]:
# Create an empty list 
new_list =[] 
# Iterate over each row 
for rows in df_word_frequency_n[:10].itertuples(): 
    # Create list for the current row 
    my_list =[rows.word, rows.frequency] 
      
    # append the list to the final list 
    new_list.append(my_list) 
  
# Print the list 
print(new_list) 

[['interior', 147207], ['replace', 144060], ['existing', 128984], ['btu', 119145], ['original', 114871], ['floor', 107525], ['gas', 106455], ['water', 105438], ['inches', 101077], ['plans', 96810]]


In [22]:
ABELL = ['ABELL']
ALLENDALE = ['ALLENDALE']
ARCADIA = ['ARCADIA']
ARLINGTON = ['ARLINGTON']
ARMISTEADGARDENS = ['ARMISTEAD GARDENS']
ASHBURTON = ['ASHBURTON']
BALTIMOREHIGHLANDS = ['BALTIMORE HIGHLANDS']
BALTIMORELINWOOD = ['BALTIMORE-LINWOOD']
BARCLAY = ['BARCLAY']
BARRECIRCLE = ['BARRE CIRCLE']
BEECHFIELD = ['BEECHFIELD']
BELAIREDISON = ['BELAIR-EDISON']
BELAIRPARKSIDE = ['BELAIR-PARKSIDE']
BELLONAGITTINGS = ['BELLONA-GITTINGS']
BEREA = ['BEREA']
BETTERWAVERLY = ['BETTER WAVERLY']
BEVERLYHILLS = ['BEVERLY HILLS']
BIDDLESTREET = ['BIDDLE STREET']
BLYTHEWOOD = ['BLYTHEWOOD']
BOLTONHILL = ['BOLTON HILL']
BOOTHBOYD = ['BOOTH-BOYD']
BREWERSHILL = ['BREWERS HILL']
BRIDGEVIEWGREENLAWN = ['BRIDGEVIEW/GREENLAWN']
BROADWAYEAST = ['BROADWAY EAST']
BROENINGMANOR = ['BROENING MANOR']
BROOKLYN = ['BROOKLYN']
BURLEITHLEIGHTON = ['BURLEITH-LEIGHTON']
#BUTCHERSHILL = ['BUTCHER'S HILL']
CALLAWAYGARRISON = ['CALLAWAY-GARRISON']
CAMERONVILLAGE = ['CAMERON VILLAGE']
CANTON = ['CANTON']
CANTONINDUSTRIALAREA = ['CANTON INDUSTRIAL AREA']
CARROLLCAMDENINDUSTRIALAR = ['CARROLL - CAMDEN INDUSTRIAL AR']
CARROLLPARK = ['CARROLL PARK']
CARROLLSOUTHHILTON = ['CARROLL-SOUTH HILTON']
CARROLLTONRIDGE = ['CARROLLTON RIDGE']
CEDARCROFT = ['CEDARCROFT']
CEDMONT = ['CEDMONT']
CEDONIA = ['CEDONIA']
CENTRALFORESTPARK = ['CENTRAL FOREST PARK']
CENTRALPARKHEIGHTS = ['CENTRAL PARK HEIGHTS']
CHARLESNORTH = ['CHARLES NORTH']
CHARLESVILLAGE = ['CHARLES VILLAGE']
CHERRYHILL = ['CHERRY HILL']
CHESWOLDE = ['CHESWOLDE']
CHINQUAPINPARKBELVEDERE = ['CHINQUAPIN PARK-BELVEDERE']
CHRISTOPHER = ['CHRISTOPHER']
CLAREMONT = ['CLAREMONT']
CLAREMONTFREEDOM = ['CLAREMONT-FREEDOM']
CLIFTONPARK = ['CLIFTON PARK']
COLDSPRING = ['COLDSPRING']
COLDSTREAMHOMESTEADMONTEBELL = ['COLDSTREAM HOMESTEAD MONTEBELL']
CONCERNEDCITIZENSOFFORESTP = ['CONCERNED CITIZENS OF FOREST P']
COPPINHEIGHTSASHCOEAST = ['COPPIN HEIGHTS/ASH-CO-EAST']
CROSSCOUNTRY = ['CROSS COUNTRY']
CROSSKEYS = ['CROSS KEYS']
CURTISBAY = ['CURTIS BAY']
CURTISBAYINDUSTRIAL = ['CURTIS BAY INDUSTRIAL']
CYLBURN = ['CYLBURN']
DARLEYPARK = ['DARLEY PARK']
DICKEYVILLE = ['DICKEYVILLE']
DOLFIELD = ['DOLFIELD']
DORCHESTER = ['DORCHESTER']
DOWNTOWN = ['DOWNTOWN']
DRUIDHEIGHTS = ['DRUID HEIGHTS']
DRUIDHILLPARK = ['DRUID HILL PARK']
DUNBARBROADWAY = ['DUNBAR-BROADWAY']
DUNDALKMARINETERMINAL = ['DUNDALK MARINE TERMINAL']
EASTARLINGTON = ['EAST ARLINGTON']
EASTBALTIMOREMIDWAY = ['EAST BALTIMORE MIDWAY']
EASTERWOOD = ['EASTERWOOD']
EASTWOOD = ['EASTWOOD']
EDMONDSONVILLAGE = ['EDMONDSON VILLAGE']
EDNORGARDENSLAKESIDE = ['EDNOR GARDENS-LAKESIDE']
ELLWOODPARKMONUMENT = ['ELLWOOD PARK/MONUMENT']
EVERGREEN = ['EVERGREEN']
EVERGREENLAWN = ['EVERGREEN LAWN']
FAIRFIELDAREA = ['FAIRFIELD AREA']
FAIRMONT = ['FAIRMONT']
FALLSTAFF = ['FALLSTAFF']
FEDERALHILL = ['FEDERAL HILL']
FEDERALHILLMONTGOMERY = ['FEDERAL HILL/MONTGOMERY']
FELLSPOINT = ['FELLS POINT']
FORESTPARK = ['FOREST PARK']
FORESTPARKGOLFCOURSE = ['FOREST PARK GOLF COURSE']
FOURBYFOUR = ['FOUR BY FOUR']
FRANKFORD = ['FRANKFORD']
FRANKLINSQUARE = ['FRANKLIN SQUARE']
FRANKLINTOWN = ['FRANKLINTOWN']
FRANKLINTOWNROAD = ['FRANKLINTOWN ROAD']
GARWYNOAKS = ['GARWYN OAKS']
GAYSTREET = ['GAY STREET']
GLEN = ['GLEN']
GLENOAKS = ['GLEN OAKS']
GLENHAMBELFORD = ['GLENHAM-BELFORD']
GRACELANDPARK = ['GRACELAND PARK']
GREEKTOWN = ['GREEKTOWN']
GREENMOUNTCEMETERY = ['GREENMOUNT CEMETERY']
GREENMOUNTWEST = ['GREENMOUNT WEST']
GREENSPRING = ['GREENSPRING']
GROVEPARK = ['GROVE PARK']
GUILFORD = ['GUILFORD']
GWYNNSFALLS = ['GWYNNS FALLS']
GWYNNSFALLSLEAKINPARK = ['GWYNNS FALLS/LEAKIN PARK']
HAMPDEN = ['HAMPDEN']
HANLONLONGWOOD = ['HANLON-LONGWOOD']
HARFORDECHODALEPERRINGPARKW = ['HARFORD-ECHODALE/PERRING PARKW']
HARLEMPARK = ['HARLEM PARK']
HARWOOD = ['HARWOOD']
HAWKINSPOINT = ['HAWKINS POINT']
HERITAGECROSSING = ['HERITAGE CROSSING']
HIGHLANDTOWN = ['HIGHLANDTOWN']
HILLEN = ['HILLEN']
HOESHEIGHTS = ['HOES HEIGHTS']
HOLABIRDINDUSTRIALPARK = ['HOLABIRD INDUSTRIAL PARK']
HOLLINSMARKET = ['HOLLINS MARKET']
HOMELAND = ['HOMELAND']
HOPKINSBAYVIEW = ['HOPKINS BAYVIEW']
HOWARDPARK = ['HOWARD PARK']
HUNTINGRIDGE = ['HUNTING RIDGE']
IDLEWOOD = ['IDLEWOOD']
INNERHARBOR = ['INNER HARBOR']
IRVINGTON = ['IRVINGTON']
JOHNSHOPKINSHOMEWOOD = ['JOHNS HOPKINS HOMEWOOD']
JOHNSTONSQUARE = ['JOHNSTON SQUARE']
JONESFALLSAREA = ['JONES FALLS AREA']
JONESTOWN = ['JONESTOWN']
JOSEPHLEE = ['JOSEPH LEE']
KENILWORTHPARK = ['KENILWORTH PARK']
KERNEWOOD = ['KERNEWOOD']
KESWICK = ['KESWICK']
KRESSON = ['KRESSON']
LAFAYETTE = ['LAFAYETTE']
LAKEEVESHAM = ['LAKE EVESHAM']
LAKEWALKER = ['LAKE WALKER']
LAKELAND = ['LAKELAND']
LANGSTONHUGHES = ['LANGSTON HUGHES']
LAURAVILLE = ['LAURAVILLE']
LEVINDALE = ['LEVINDALE']
LIBERTYSQUARE = ['LIBERTY SQUARE']
LITTLEITALY = ['LITTLE ITALY']
LOCHRAVEN = ['LOCH RAVEN']
LOCUSTPOINT = ['LOCUST POINT']
LOCUSTPOINTINDUSTRIALAREA = ['LOCUST POINT INDUSTRIAL AREA']
LOWERHERRINGRUNPARK = ['LOWER HERRING RUN PARK']
LOYOLANOTREDAME = ['LOYOLA/NOTRE DAME']
LUCILLEPARK = ['LUCILLE PARK']
MADISONPARK = ['MADISON PARK']
MADISONEASTEND = ['MADISON-EASTEND']
MAYFIELD = ['MAYFIELD']
MCELDERRYPARK = ['MCELDERRY PARK']
MEDFIELD = ['MEDFIELD']
MEDFORD = ['MEDFORD']
MIDDLEBRANCHREEDBIRDPARKS = ['MIDDLE BRANCH/REEDBIRD PARKS']
MIDDLEEAST = ['MIDDLE EAST']
MIDGOVANS = ['MID-GOVANS']
MIDTOWNBELVEDERE = ['MID-TOWN BELVEDERE']
MIDTOWNEDMONDSON = ['MIDTOWN-EDMONDSON']
MILLHILL = ['MILLHILL']
MILTONMONTFORD = ['MILTON-MONTFORD']
MONDAWMIN = ['MONDAWMIN']
MONTEBELLO = ['MONTEBELLO']
MORAVIAWALTHER = ['MORAVIA-WALTHER']
MORGANPARK = ['MORGAN PARK']
MORGANSTATEUNIVERSITY = ['MORGAN STATE UNIVERSITY']
MORRELLPARK = ['MORRELL PARK']
MOSHER = ['MOSHER']
MOUNTHOLLY = ['MOUNT HOLLY']
MOUNTVERNON = ['MOUNT VERNON']
MOUNTWASHINGTON = ['MOUNT WASHINGTON']
MOUNTWINANS = ['MOUNT WINANS']
MTPLEASANTPARK = ['MT PLEASANT PARK']
NEWNORTHWOOD = ['NEW NORTHWOOD']
NEWSOUTHWESTMOUNTCLARE = ['NEW SOUTHWEST/MOUNT CLARE']
NORTHHARFORDROAD = ['NORTH HARFORD ROAD']
NORTHROLANDPARKPOPLARHILL = ['NORTH ROLAND PARK/POPLAR HILL']
NORTHWESTCOMMUNITYACTION = ['NORTHWEST COMMUNITY ACTION']
OAKENSHAWE = ['OAKENSHAWE']
OAKLEE = ['OAKLEE']
#ODONNELLHEIGHTS = ['O'DONNELL HEIGHTS']
OLDTOWN = ['OLDTOWN']
OLIVER = ['OLIVER']
ORANGEVILLE = ['ORANGEVILLE']
ORANGEVILLEINDUSTRIALAREA = ['ORANGEVILLE INDUSTRIAL AREA']
ORIGINALNORTHWOOD = ['ORIGINAL NORTHWOOD']
OTTERBEIN = ['OTTERBEIN']
OVERLEA = ['OVERLEA']
PANWAYBRADDISHAVENUE = ['PANWAY/BRADDISH AVENUE']
PARKCIRCLE = ['PARK CIRCLE']
PARKLANE = ['PARKLANE']
PARKSIDE = ['PARKSIDE']
PARKVIEWWOODBROOK = ['PARKVIEW/WOODBROOK']
PATTERSONPARK = ['PATTERSON PARK']
PATTERSONPLACE = ['PATTERSON PLACE']
PENLUCY = ['PEN LUCY']
PENNNORTH = ['PENN NORTH']
PENNFALLSWAY = ['PENN-FALLSWAY']
PENROSE = ['PENROSE']
PENROSEFAYETTESTREETOUTREAC = ['PENROSE/FAYETTE STREET OUTREAC']
PERKINSHOMES = ['PERKINS HOMES']
PERRINGLOCH = ['PERRING LOCH']
PIMLICOGOODNEIGHBORS = ['PIMLICO GOOD NEIGHBORS']
PLEASANTVIEWGARDENS = ['PLEASANT VIEW GARDENS']
POPPLETON = ['POPPLETON']
PORTCOVINGTON = ['PORT COVINGTON']
PULASKIINDUSTRIALAREA = ['PULASKI INDUSTRIAL AREA']
PURNELL = ['PURNELL']
RADNORWINSTON = ['RADNOR-WINSTON']
RAMBLEWOOD = ['RAMBLEWOOD']
REISTERSTOWNSTATION = ['REISTERSTOWN STATION']
REMINGTON = ['REMINGTON']
RESERVOIRHILL = ['RESERVOIR HILL']
RICHNORSPRINGS = ['RICHNOR SPRINGS']
#RIDGELYSDELIGHT = ['RIDGELY'S DELIGHT']
RIVERSIDE = ['RIVERSIDE']
ROGNELHEIGHTS = ['ROGNEL HEIGHTS']
ROLANDPARK = ['ROLAND PARK']
ROLANDPARKPOPLARHILL = ['ROLAND PARK/POPLAR HILL']
ROSEBANK = ['ROSEBANK']
ROSEMONT = ['ROSEMONT']
ROSEMONTEAST = ['ROSEMONT EAST']
ROSEMONTHOMEOWNERSTENANTS = ['ROSEMONT HOMEOWNERS/TENANTS']
SABINAMATTFELDT = ['SABINA-MATTFELDT']
SAINTAGNES = ['SAINT AGNES']
SAINTHELENA = ['SAINT HELENA']
SAINTJOSEPHS = ['SAINT JOSEPHS']
SAINTPAUL = ['SAINT PAUL']
SANDTOWNWINCHESTER = ['SANDTOWN-WINCHESTER']
SBIC = ['SBIC']
SETONBUSINESSPARK = ['SETON BUSINESS PARK']
SETONHILL = ['SETON HILL']
SHARPLEADENHALL = ['SHARP-LEADENHALL']
SHIPLEYHILL = ['SHIPLEY HILL']
SOUTHCLIFTONPARK = ['SOUTH CLIFTON PARK']
SPRINGGARDENINDUSTRIALAREA = ['SPRING GARDEN INDUSTRIAL AREA']
STADIUMAREA = ['STADIUM AREA']
STONEWOODPENTWOODWINSTON = ['STONEWOOD-PENTWOOD-WINSTON']
TAYLORHEIGHTS = ['TAYLOR HEIGHTS']
TENHILLS = ['TEN HILLS']
THEORCHARDS = ['THE ORCHARDS']
TOWANDAGRANTLEY = ['TOWANDA-GRANTLEY']
TREMONT = ['TREMONT']
TUSCANYCANTERBURY = ['TUSCANY-CANTERBURY']
UNIONSQUARE = ['UNION SQUARE']
UNIVERSITYOFMARYLAND = ['UNIVERSITY OF MARYLAND']
UPLANDS = ['UPLANDS']
UPPERFELLSPOINT = ['UPPER FELLS POINT']
UPTON = ['UPTON']
VILLAGESOFHOMELAND = ['VILLAGES OF HOMELAND']
VIOLETVILLE = ['VIOLETVILLE']
WAKEFIELD = ['WAKEFIELD']
WALBROOK = ['WALBROOK']
WALTHERSON = ['WALTHERSON']
WASHINGTONHILL = ['WASHINGTON HILL']
WASHINGTONVILLAGE = ['WASHINGTON VILLAGE']
WAVERLY = ['WAVERLY']
WESTARLINGTON = ['WEST ARLINGTON']
WESTFORESTPARK = ['WEST FOREST PARK']
WESTHILLS = ['WEST HILLS']
WESTFIELD = ['WESTFIELD']
WESTGATE = ['WESTGATE']
WESTPORT = ['WESTPORT']
WILSONHEIGHTS = ['WILSON HEIGHTS']
WILSONPARK = ['WILSON PARK']
WINCHESTER = ['WINCHESTER']
WINDSORHILLS = ['WINDSOR HILLS']
WINSTONGOVANS = ['WINSTON-GOVANS']
WOODBERRY = ['WOODBERRY']
WOODBOURNEHEIGHTS = ['WOODBOURNE HEIGHTS']
WOODBOURNEMCCABE = ['WOODBOURNE-MCCABE']
WOODMERE = ['WOODMERE']
WOODRING = ['WOODRING']
WYMANPARK = ['WYMAN PARK']
WYNDHURST = ['WYNDHURST']
YALEHEIGHTS = ['YALE HEIGHTS']

# selecting rows based on condition 
ABELL_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(ABELL)]
ALLENDALE_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(ALLENDALE)]
ARCADIA_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(ARCADIA)]
ARLINGTON_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(ARLINGTON)]
ARMISTEADGARDENS_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(ARMISTEADGARDENS)]
ASHBURTON_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(ASHBURTON)]
BALTIMOREHIGHLANDS_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(BALTIMOREHIGHLANDS)]
BALTIMORELINWOOD_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(BALTIMORELINWOOD)]
BARCLAY_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(BARCLAY)]
BARRECIRCLE_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(BARRECIRCLE)]
BEECHFIELD_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(BEECHFIELD)]
BELAIREDISON_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(BELAIREDISON)]
BELAIRPARKSIDE_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(BELAIRPARKSIDE)]
BELLONAGITTINGS_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(BELLONAGITTINGS)]
BEREA_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(BEREA)]
BETTERWAVERLY_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(BETTERWAVERLY)]
BEVERLYHILLS_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(BEVERLYHILLS)]
BIDDLESTREET_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(BIDDLESTREET)]
BLYTHEWOOD_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(BLYTHEWOOD)]
BOLTONHILL_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(BOLTONHILL)]
BOOTHBOYD_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(BOOTHBOYD)]
BREWERSHILL_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(BREWERSHILL)]
BRIDGEVIEWGREENLAWN_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(BRIDGEVIEWGREENLAWN)]
BROADWAYEAST_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(BROADWAYEAST)]
BROENINGMANOR_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(BROENINGMANOR)]
BROOKLYN_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(BROOKLYN)]
BURLEITHLEIGHTON_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(BURLEITHLEIGHTON)]
#BUTCHERSHILL_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(BUTCHERSHILL)]
CALLAWAYGARRISON_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(CALLAWAYGARRISON)]
CAMERONVILLAGE_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(CAMERONVILLAGE)]
CANTON_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(CANTON)]
CANTONINDUSTRIALAREA_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(CANTONINDUSTRIALAREA)]
CARROLLCAMDENINDUSTRIALAR_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(CARROLLCAMDENINDUSTRIALAR)]
CARROLLPARK_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(CARROLLPARK)]
CARROLLSOUTHHILTON_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(CARROLLSOUTHHILTON)]
CARROLLTONRIDGE_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(CARROLLTONRIDGE)]
CEDARCROFT_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(CEDARCROFT)]
CEDMONT_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(CEDMONT)]
CEDONIA_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(CEDONIA)]
CENTRALFORESTPARK_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(CENTRALFORESTPARK)]
CENTRALPARKHEIGHTS_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(CENTRALPARKHEIGHTS)]
CHARLESNORTH_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(CHARLESNORTH)]
CHARLESVILLAGE_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(CHARLESVILLAGE)]
CHERRYHILL_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(CHERRYHILL)]
CHESWOLDE_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(CHESWOLDE)]
CHINQUAPINPARKBELVEDERE_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(CHINQUAPINPARKBELVEDERE)]
CHRISTOPHER_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(CHRISTOPHER)]
CLAREMONT_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(CLAREMONT)]
CLAREMONTFREEDOM_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(CLAREMONTFREEDOM)]
CLIFTONPARK_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(CLIFTONPARK)]
COLDSPRING_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(COLDSPRING)]
COLDSTREAMHOMESTEADMONTEBELL_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(COLDSTREAMHOMESTEADMONTEBELL)]
CONCERNEDCITIZENSOFFORESTP_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(CONCERNEDCITIZENSOFFORESTP)]
COPPINHEIGHTSASHCOEAST_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(COPPINHEIGHTSASHCOEAST)]
CROSSCOUNTRY_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(CROSSCOUNTRY)]
CROSSKEYS_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(CROSSKEYS)]
CURTISBAY_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(CURTISBAY)]
CURTISBAYINDUSTRIAL_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(CURTISBAYINDUSTRIAL)]
CYLBURN_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(CYLBURN)]
DARLEYPARK_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(DARLEYPARK)]
DICKEYVILLE_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(DICKEYVILLE)]
DOLFIELD_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(DOLFIELD)]
DORCHESTER_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(DORCHESTER)]
DOWNTOWN_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(DOWNTOWN)]
DRUIDHEIGHTS_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(DRUIDHEIGHTS)]
DRUIDHILLPARK_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(DRUIDHILLPARK)]
DUNBARBROADWAY_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(DUNBARBROADWAY)]
DUNDALKMARINETERMINAL_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(DUNDALKMARINETERMINAL)]
EASTARLINGTON_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(EASTARLINGTON)]
EASTBALTIMOREMIDWAY_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(EASTBALTIMOREMIDWAY)]
EASTERWOOD_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(EASTERWOOD)]
EASTWOOD_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(EASTWOOD)]
EDMONDSONVILLAGE_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(EDMONDSONVILLAGE)]
EDNORGARDENSLAKESIDE_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(EDNORGARDENSLAKESIDE)]
ELLWOODPARKMONUMENT_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(ELLWOODPARKMONUMENT)]
EVERGREEN_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(EVERGREEN)]
EVERGREENLAWN_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(EVERGREENLAWN)]
FAIRFIELDAREA_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(FAIRFIELDAREA)]
FAIRMONT_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(FAIRMONT)]
FALLSTAFF_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(FALLSTAFF)]
FEDERALHILL_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(FEDERALHILL)]
FEDERALHILLMONTGOMERY_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(FEDERALHILLMONTGOMERY)]
FELLSPOINT_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(FELLSPOINT)]
FORESTPARK_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(FORESTPARK)]
FORESTPARKGOLFCOURSE_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(FORESTPARKGOLFCOURSE)]
FOURBYFOUR_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(FOURBYFOUR)]
FRANKFORD_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(FRANKFORD)]
FRANKLINSQUARE_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(FRANKLINSQUARE)]
FRANKLINTOWN_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(FRANKLINTOWN)]
FRANKLINTOWNROAD_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(FRANKLINTOWNROAD)]
GARWYNOAKS_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(GARWYNOAKS)]
GAYSTREET_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(GAYSTREET)]
GLEN_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(GLEN)]
GLENOAKS_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(GLENOAKS)]
GLENHAMBELFORD_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(GLENHAMBELFORD)]
GRACELANDPARK_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(GRACELANDPARK)]
GREEKTOWN_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(GREEKTOWN)]
GREENMOUNTCEMETERY_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(GREENMOUNTCEMETERY)]
GREENMOUNTWEST_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(GREENMOUNTWEST)]
GREENSPRING_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(GREENSPRING)]
GROVEPARK_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(GROVEPARK)]
GUILFORD_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(GUILFORD)]
GWYNNSFALLS_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(GWYNNSFALLS)]
GWYNNSFALLSLEAKINPARK_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(GWYNNSFALLSLEAKINPARK)]
HAMPDEN_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(HAMPDEN)]
HANLONLONGWOOD_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(HANLONLONGWOOD)]
HARFORDECHODALEPERRINGPARKW_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(HARFORDECHODALEPERRINGPARKW)]
HARLEMPARK_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(HARLEMPARK)]
HARWOOD_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(HARWOOD)]
HAWKINSPOINT_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(HAWKINSPOINT)]
HERITAGECROSSING_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(HERITAGECROSSING)]
HIGHLANDTOWN_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(HIGHLANDTOWN)]
HILLEN_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(HILLEN)]
HOESHEIGHTS_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(HOESHEIGHTS)]
HOLABIRDINDUSTRIALPARK_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(HOLABIRDINDUSTRIALPARK)]
HOLLINSMARKET_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(HOLLINSMARKET)]
HOMELAND_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(HOMELAND)]
HOPKINSBAYVIEW_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(HOPKINSBAYVIEW)]
HOWARDPARK_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(HOWARDPARK)]
HUNTINGRIDGE_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(HUNTINGRIDGE)]
IDLEWOOD_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(IDLEWOOD)]
INNERHARBOR_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(INNERHARBOR)]
IRVINGTON_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(IRVINGTON)]
JOHNSHOPKINSHOMEWOOD_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(JOHNSHOPKINSHOMEWOOD)]
JOHNSTONSQUARE_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(JOHNSTONSQUARE)]
JONESFALLSAREA_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(JONESFALLSAREA)]
JONESTOWN_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(JONESTOWN)]
JOSEPHLEE_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(JOSEPHLEE)]
KENILWORTHPARK_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(KENILWORTHPARK)]
KERNEWOOD_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(KERNEWOOD)]
KESWICK_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(KESWICK)]
KRESSON_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(KRESSON)]
LAFAYETTE_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(LAFAYETTE)]
LAKEEVESHAM_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(LAKEEVESHAM)]
LAKEWALKER_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(LAKEWALKER)]
LAKELAND_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(LAKELAND)]
LANGSTONHUGHES_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(LANGSTONHUGHES)]
LAURAVILLE_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(LAURAVILLE)]
LEVINDALE_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(LEVINDALE)]
LIBERTYSQUARE_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(LIBERTYSQUARE)]
LITTLEITALY_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(LITTLEITALY)]
LOCHRAVEN_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(LOCHRAVEN)]
LOCUSTPOINT_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(LOCUSTPOINT)]
LOCUSTPOINTINDUSTRIALAREA_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(LOCUSTPOINTINDUSTRIALAREA)]
LOWERHERRINGRUNPARK_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(LOWERHERRINGRUNPARK)]
LOYOLANOTREDAME_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(LOYOLANOTREDAME)]
LUCILLEPARK_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(LUCILLEPARK)]
MADISONPARK_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(MADISONPARK)]
MADISONEASTEND_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(MADISONEASTEND)]
MAYFIELD_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(MAYFIELD)]
MCELDERRYPARK_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(MCELDERRYPARK)]
MEDFIELD_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(MEDFIELD)]
MEDFORD_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(MEDFORD)]
MIDDLEBRANCHREEDBIRDPARKS_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(MIDDLEBRANCHREEDBIRDPARKS)]
MIDDLEEAST_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(MIDDLEEAST)]
MIDGOVANS_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(MIDGOVANS)]
MIDTOWNBELVEDERE_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(MIDTOWNBELVEDERE)]
MIDTOWNEDMONDSON_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(MIDTOWNEDMONDSON)]
MILLHILL_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(MILLHILL)]
MILTONMONTFORD_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(MILTONMONTFORD)]
MONDAWMIN_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(MONDAWMIN)]
MONTEBELLO_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(MONTEBELLO)]
MORAVIAWALTHER_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(MORAVIAWALTHER)]
MORGANPARK_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(MORGANPARK)]
MORGANSTATEUNIVERSITY_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(MORGANSTATEUNIVERSITY)]
MORRELLPARK_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(MORRELLPARK)]
MOSHER_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(MOSHER)]
MOUNTHOLLY_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(MOUNTHOLLY)]
MOUNTVERNON_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(MOUNTVERNON)]
MOUNTWASHINGTON_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(MOUNTWASHINGTON)]
MOUNTWINANS_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(MOUNTWINANS)]
MTPLEASANTPARK_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(MTPLEASANTPARK)]
NEWNORTHWOOD_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(NEWNORTHWOOD)]
NEWSOUTHWESTMOUNTCLARE_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(NEWSOUTHWESTMOUNTCLARE)]
NORTHHARFORDROAD_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(NORTHHARFORDROAD)]
NORTHROLANDPARKPOPLARHILL_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(NORTHROLANDPARKPOPLARHILL)]
NORTHWESTCOMMUNITYACTION_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(NORTHWESTCOMMUNITYACTION)]
OAKENSHAWE_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(OAKENSHAWE)]
OAKLEE_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(OAKLEE)]
#ODONNELLHEIGHTS_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(ODONNELLHEIGHTS)]
OLDTOWN_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(OLDTOWN)]
OLIVER_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(OLIVER)]
ORANGEVILLE_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(ORANGEVILLE)]
ORANGEVILLEINDUSTRIALAREA_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(ORANGEVILLEINDUSTRIALAREA)]
ORIGINALNORTHWOOD_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(ORIGINALNORTHWOOD)]
OTTERBEIN_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(OTTERBEIN)]
OVERLEA_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(OVERLEA)]
PANWAYBRADDISHAVENUE_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(PANWAYBRADDISHAVENUE)]
PARKCIRCLE_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(PARKCIRCLE)]
PARKLANE_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(PARKLANE)]
PARKSIDE_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(PARKSIDE)]
PARKVIEWWOODBROOK_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(PARKVIEWWOODBROOK)]
PATTERSONPARK_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(PATTERSONPARK)]
PATTERSONPLACE_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(PATTERSONPLACE)]
PENLUCY_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(PENLUCY)]
PENNNORTH_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(PENNNORTH)]
PENNFALLSWAY_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(PENNFALLSWAY)]
PENROSE_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(PENROSE)]
PENROSEFAYETTESTREETOUTREAC_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(PENROSEFAYETTESTREETOUTREAC)]
PERKINSHOMES_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(PERKINSHOMES)]
PERRINGLOCH_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(PERRINGLOCH)]
PIMLICOGOODNEIGHBORS_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(PIMLICOGOODNEIGHBORS)]
PLEASANTVIEWGARDENS_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(PLEASANTVIEWGARDENS)]
POPPLETON_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(POPPLETON)]
PORTCOVINGTON_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(PORTCOVINGTON)]
PULASKIINDUSTRIALAREA_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(PULASKIINDUSTRIALAREA)]
PURNELL_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(PURNELL)]
RADNORWINSTON_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(RADNORWINSTON)]
RAMBLEWOOD_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(RAMBLEWOOD)]
REISTERSTOWNSTATION_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(REISTERSTOWNSTATION)]
REMINGTON_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(REMINGTON)]
RESERVOIRHILL_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(RESERVOIRHILL)]
RICHNORSPRINGS_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(RICHNORSPRINGS)]
#RIDGELYSDELIGHT_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(RIDGELYSDELIGHT)]
RIVERSIDE_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(RIVERSIDE)]
ROGNELHEIGHTS_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(ROGNELHEIGHTS)]
ROLANDPARK_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(ROLANDPARK)]
ROLANDPARKPOPLARHILL_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(ROLANDPARKPOPLARHILL)]
ROSEBANK_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(ROSEBANK)]
ROSEMONT_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(ROSEMONT)]
ROSEMONTEAST_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(ROSEMONTEAST)]
ROSEMONTHOMEOWNERSTENANTS_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(ROSEMONTHOMEOWNERSTENANTS)]
SABINAMATTFELDT_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(SABINAMATTFELDT)]
SAINTAGNES_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(SAINTAGNES)]
SAINTHELENA_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(SAINTHELENA)]
SAINTJOSEPHS_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(SAINTJOSEPHS)]
SAINTPAUL_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(SAINTPAUL)]
SANDTOWNWINCHESTER_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(SANDTOWNWINCHESTER)]
SBIC_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(SBIC)]
SETONBUSINESSPARK_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(SETONBUSINESSPARK)]
SETONHILL_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(SETONHILL)]
SHARPLEADENHALL_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(SHARPLEADENHALL)]
SHIPLEYHILL_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(SHIPLEYHILL)]
SOUTHCLIFTONPARK_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(SOUTHCLIFTONPARK)]
SPRINGGARDENINDUSTRIALAREA_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(SPRINGGARDENINDUSTRIALAREA)]
STADIUMAREA_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(STADIUMAREA)]
STONEWOODPENTWOODWINSTON_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(STONEWOODPENTWOODWINSTON)]
TAYLORHEIGHTS_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(TAYLORHEIGHTS)]
TENHILLS_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(TENHILLS)]
THEORCHARDS_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(THEORCHARDS)]
TOWANDAGRANTLEY_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(TOWANDAGRANTLEY)]
TREMONT_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(TREMONT)]
TUSCANYCANTERBURY_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(TUSCANYCANTERBURY)]
UNIONSQUARE_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(UNIONSQUARE)]
UNIVERSITYOFMARYLAND_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(UNIVERSITYOFMARYLAND)]
UPLANDS_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(UPLANDS)]
UPPERFELLSPOINT_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(UPPERFELLSPOINT)]
UPTON_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(UPTON)]
VILLAGESOFHOMELAND_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(VILLAGESOFHOMELAND)]
VIOLETVILLE_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(VIOLETVILLE)]
WAKEFIELD_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(WAKEFIELD)]
WALBROOK_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(WALBROOK)]
WALTHERSON_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(WALTHERSON)]
WASHINGTONHILL_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(WASHINGTONHILL)]
WASHINGTONVILLAGE_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(WASHINGTONVILLAGE)]
WAVERLY_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(WAVERLY)]
WESTARLINGTON_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(WESTARLINGTON)]
WESTFORESTPARK_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(WESTFORESTPARK)]
WESTHILLS_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(WESTHILLS)]
WESTFIELD_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(WESTFIELD)]
WESTGATE_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(WESTGATE)]
WESTPORT_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(WESTPORT)]
WILSONHEIGHTS_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(WILSONHEIGHTS)]
WILSONPARK_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(WILSONPARK)]
WINCHESTER_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(WINCHESTER)]
WINDSORHILLS_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(WINDSORHILLS)]
WINSTONGOVANS_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(WINSTONGOVANS)]
WOODBERRY_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(WOODBERRY)]
WOODBOURNEHEIGHTS_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(WOODBOURNEHEIGHTS)]
WOODBOURNEMCCABE_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(WOODBOURNEMCCABE)]
WOODMERE_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(WOODMERE)]
WOODRING_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(WOODRING)]
WYMANPARK_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(WYMANPARK)]
WYNDHURST_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(WYNDHURST)]
YALEHEIGHTS_df = house_permit_value_filter.loc[house_permit_value_filter['Neighborhood'].isin(YALEHEIGHTS)]

NameError: name 'house_permit_value_filter' is not defined

In [ ]:
dflist = [ABELL_df,
ALLENDALE_df,
ARCADIA_df,
ARLINGTON_df,
ARMISTEADGARDENS_df,
ASHBURTON_df,
BALTIMOREHIGHLANDS_df,
BALTIMORELINWOOD_df,
BARCLAY_df,
BARRECIRCLE_df,
BEECHFIELD_df,
BELAIREDISON_df,
BELAIRPARKSIDE_df,
BELLONAGITTINGS_df,
BEREA_df,
BETTERWAVERLY_df,
BEVERLYHILLS_df,
BIDDLESTREET_df,
BLYTHEWOOD_df,
BOLTONHILL_df,
BOOTHBOYD_df,
BREWERSHILL_df,
BRIDGEVIEWGREENLAWN_df,
BROADWAYEAST_df,
BROENINGMANOR_df,
BROOKLYN_df,
BURLEITHLEIGHTON_df,
CALLAWAYGARRISON_df,
CAMERONVILLAGE_df,
CANTON_df,
CANTONINDUSTRIALAREA_df,
CARROLLCAMDENINDUSTRIALAR_df,
CARROLLPARK_df,
CARROLLSOUTHHILTON_df,
CARROLLTONRIDGE_df,
CEDARCROFT_df,
CEDMONT_df,
CEDONIA_df,
CENTRALFORESTPARK_df,
CENTRALPARKHEIGHTS_df,
CHARLESNORTH_df,
CHARLESVILLAGE_df,
CHERRYHILL_df,
CHESWOLDE_df,
CHINQUAPINPARKBELVEDERE_df,
CHRISTOPHER_df,
CLAREMONT_df,
CLAREMONTFREEDOM_df,
CLIFTONPARK_df,
COLDSPRING_df,
COLDSTREAMHOMESTEADMONTEBELL_df,
CONCERNEDCITIZENSOFFORESTP_df,
COPPINHEIGHTSASHCOEAST_df,
CROSSCOUNTRY_df,
CROSSKEYS_df,
CURTISBAY_df,
CURTISBAYINDUSTRIAL_df,
CYLBURN_df,
DARLEYPARK_df,
DICKEYVILLE_df,
DOLFIELD_df,
DORCHESTER_df,
DOWNTOWN_df,
DRUIDHEIGHTS_df,
DRUIDHILLPARK_df,
DUNBARBROADWAY_df,
DUNDALKMARINETERMINAL_df,
EASTARLINGTON_df,
EASTBALTIMOREMIDWAY_df,
EASTERWOOD_df,
EASTWOOD_df,
EDMONDSONVILLAGE_df,
EDNORGARDENSLAKESIDE_df,
ELLWOODPARKMONUMENT_df,
EVERGREEN_df,
EVERGREENLAWN_df,
FAIRFIELDAREA_df,
FAIRMONT_df,
FALLSTAFF_df,
FEDERALHILL_df,
FEDERALHILLMONTGOMERY_df,
FELLSPOINT_df,
FORESTPARK_df,
FORESTPARKGOLFCOURSE_df,
FOURBYFOUR_df,
FRANKFORD_df,
FRANKLINSQUARE_df,
FRANKLINTOWN_df,
FRANKLINTOWNROAD_df,
GARWYNOAKS_df,
GAYSTREET_df,
GLEN_df,
GLENOAKS_df,
GLENHAMBELFORD_df,
GRACELANDPARK_df,
GREEKTOWN_df,
GREENMOUNTCEMETERY_df,
GREENMOUNTWEST_df,
GREENSPRING_df,
GROVEPARK_df,
GUILFORD_df,
GWYNNSFALLS_df,
GWYNNSFALLSLEAKINPARK_df,
HAMPDEN_df,
HANLONLONGWOOD_df,
HARFORDECHODALEPERRINGPARKW_df,
HARLEMPARK_df,
HARWOOD_df,
HAWKINSPOINT_df,
HERITAGECROSSING_df,
HIGHLANDTOWN_df,
HILLEN_df,
HOESHEIGHTS_df,
HOLABIRDINDUSTRIALPARK_df,
HOLLINSMARKET_df,
HOMELAND_df,
HOPKINSBAYVIEW_df,
HOWARDPARK_df,
HUNTINGRIDGE_df,
IDLEWOOD_df,
INNERHARBOR_df,
IRVINGTON_df,
JOHNSHOPKINSHOMEWOOD_df,
JOHNSTONSQUARE_df,
JONESFALLSAREA_df,
JONESTOWN_df,
JOSEPHLEE_df,
KENILWORTHPARK_df,
KERNEWOOD_df,
KESWICK_df,
KRESSON_df,
LAFAYETTE_df,
LAKEEVESHAM_df,
LAKEWALKER_df,
LAKELAND_df,
LANGSTONHUGHES_df,
LAURAVILLE_df,
LEVINDALE_df,
LIBERTYSQUARE_df,
LITTLEITALY_df,
LOCHRAVEN_df,
LOCUSTPOINT_df,
LOCUSTPOINTINDUSTRIALAREA_df,
LOWERHERRINGRUNPARK_df,
LOYOLANOTREDAME_df,
LUCILLEPARK_df,
MADISONPARK_df,
MADISONEASTEND_df,
MAYFIELD_df,
MCELDERRYPARK_df,
MEDFIELD_df,
MEDFORD_df,
MIDDLEBRANCHREEDBIRDPARKS_df,
MIDDLEEAST_df,
MIDGOVANS_df,
MIDTOWNBELVEDERE_df,
MIDTOWNEDMONDSON_df,
MILLHILL_df,
MILTONMONTFORD_df,
MONDAWMIN_df,
MONTEBELLO_df,
MORAVIAWALTHER_df,
MORGANPARK_df,
MORGANSTATEUNIVERSITY_df,
MORRELLPARK_df,
MOSHER_df,
MOUNTHOLLY_df,
MOUNTVERNON_df,
MOUNTWASHINGTON_df,
MOUNTWINANS_df,
MTPLEASANTPARK_df,
NEWNORTHWOOD_df,
NEWSOUTHWESTMOUNTCLARE_df,
NORTHHARFORDROAD_df,
NORTHROLANDPARKPOPLARHILL_df,
NORTHWESTCOMMUNITYACTION_df,
OAKENSHAWE_df,
OAKLEE_df,
OLDTOWN_df,
OLIVER_df,
ORANGEVILLE_df,
ORANGEVILLEINDUSTRIALAREA_df,
ORIGINALNORTHWOOD_df,
OTTERBEIN_df,
OVERLEA_df,
PANWAYBRADDISHAVENUE_df,
PARKCIRCLE_df,
PARKLANE_df,
PARKSIDE_df,
PARKVIEWWOODBROOK_df,
PATTERSONPARK_df,
PATTERSONPLACE_df,
PENLUCY_df,
PENNNORTH_df,
PENNFALLSWAY_df,
PENROSE_df,
PENROSEFAYETTESTREETOUTREAC_df,
PERKINSHOMES_df,
PERRINGLOCH_df,
PIMLICOGOODNEIGHBORS_df,
PLEASANTVIEWGARDENS_df,
POPPLETON_df,
PORTCOVINGTON_df,
PULASKIINDUSTRIALAREA_df,
PURNELL_df,
RADNORWINSTON_df,
RAMBLEWOOD_df,
REISTERSTOWNSTATION_df,
REMINGTON_df,
RESERVOIRHILL_df,
RICHNORSPRINGS_df,
RIVERSIDE_df,
ROGNELHEIGHTS_df,
ROLANDPARK_df,
ROLANDPARKPOPLARHILL_df,
ROSEBANK_df,
ROSEMONT_df,
ROSEMONTEAST_df,
ROSEMONTHOMEOWNERSTENANTS_df,
SABINAMATTFELDT_df,
SAINTAGNES_df,
SAINTHELENA_df,
SAINTJOSEPHS_df,
SAINTPAUL_df,
SANDTOWNWINCHESTER_df,
SBIC_df,
SETONBUSINESSPARK_df,
SETONHILL_df,
SHARPLEADENHALL_df,
SHIPLEYHILL_df,
SOUTHCLIFTONPARK_df,
SPRINGGARDENINDUSTRIALAREA_df,
STADIUMAREA_df,
STONEWOODPENTWOODWINSTON_df,
TAYLORHEIGHTS_df,
TENHILLS_df,
THEORCHARDS_df,
TOWANDAGRANTLEY_df,
TREMONT_df,
TUSCANYCANTERBURY_df,
UNIONSQUARE_df,
UNIVERSITYOFMARYLAND_df,
UPLANDS_df,
UPPERFELLSPOINT_df,
UPTON_df,
VILLAGESOFHOMELAND_df,
VIOLETVILLE_df,
WAKEFIELD_df,
WALBROOK_df,
WALTHERSON_df,
WASHINGTONHILL_df,
WASHINGTONVILLAGE_df,
WAVERLY_df,
WESTARLINGTON_df,
WESTFORESTPARK_df,
WESTHILLS_df,
WESTFIELD_df,
WESTGATE_df,
WESTPORT_df,
WILSONHEIGHTS_df,
WILSONPARK_df,
WINCHESTER_df,
WINDSORHILLS_df,
WINSTONGOVANS_df,
WOODBERRY_df,
WOODBOURNEHEIGHTS_df,
WOODBOURNEMCCABE_df,
WOODMERE_df,
WOODRING_df,
WYMANPARK_df,
WYNDHURST_df,
YALEHEIGHTS_df]